In [1]:
import os
import re
import pickle
from typing import List, Dict, Any, Tuple, Optional
from pathlib import Path
from dataclasses import dataclass

from langchain.docstore.document import Document
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

In [2]:
import re
from typing import List, Dict, Set
from dataclasses import dataclass
import json
from collections import Counter

@dataclass
class DocumentChunk:
    """문서 청크를 나타내는 클래스"""
    content: str
    level: int  # 헤더 레벨 (1=H1, 2=H2, 3=H3, 4=H4)
    title: str
    full_path: List[str]  # 계층적 경로 (예: ["07. PNS", "PNS 상세", "Payment Notification"])
    metadata: Dict
    start_line: int
    end_line: int
    tags: List[str]  # 문서에서 추출된 태그들 (특수용어, 코드값, API 파라미터 등)

class TechnicalTermExtractor:
    """기술문서에서 특수 용어, 코드값, API 파라미터 등을 추출하는 클래스"""
    
    def __init__(self):
        # 기술 용어 패턴들
        self.patterns = {
            # camelCase API 파라미터 (더 정확한 패턴)
            'camel_case': re.compile(r'\b[a-z]+[A-Z][a-zA-Z0-9]*\b'),
            
            # snake_case 파라미터  
            'snake_case': re.compile(r'\b[a-z]+_[a-z0-9_]+\b'),
            
            # 모든 대문자 코드/상수 (2글자 이상)
            'all_caps_codes': re.compile(r'\b[A-Z][A-Z0-9_]{1,}\b'),
            
            # 복합 코드값 (SINGLE_PAYMENT_TRANSACTION, ONESTORECASH 등)
            'compound_codes': re.compile(r'\b[A-Z]{3,}(?:[A-Z0-9_]*[A-Z0-9]+)+\b'),
            
            # 긴 대문자 단어 (CREDITCARD, TELCOMEMBERSHIP 등)
            'long_caps_words': re.compile(r'\b[A-Z]{6,}\b'),
            
            # 약어 패턴 (2-5글자 대문자)
            'acronyms': re.compile(r'\b[A-Z]{2,5}\b'),
            
            # 코드값 패턴 확장 (MKT_ONE, 3.1.0D, OS_CODE 등)
            'code_values': re.compile(r'\b[A-Z]+_[A-Z0-9]+\b|\b\d+\.\d+\.\d+[A-Z]?\b|\b[A-Z]{2,}_[0-9]{6,}\b'),
            
            # HTTP 메서드와 상태코드
            'http_terms': re.compile(r'\b(?:GET|POST|PUT|DELETE|PATCH|HEAD|OPTIONS)\b|\b\d{3}\s+[A-Z]+\b'),
            
            # JSON 필드명 (따옴표 안의 것들)
            'json_fields': re.compile(r'"([a-zA-Z][a-zA-Z0-9_]*)"'),
            
            # 테이블 헤더의 파라미터명 (| 로 구분된 테이블에서)
            'table_params': re.compile(r'\|\s*([a-zA-Z][a-zA-Z0-9_]*)\s*\|'),
            
            # 테이블 셀의 코드값 (| 로 구분된 테이블에서)
            'table_codes': re.compile(r'\|\s*([A-Z][A-Z0-9_]{2,})\s*\|'),
            
            # 숫자가 포함된 코드 (11PAY, V21 등)
            'numeric_codes': re.compile(r'\b[A-Z]*[0-9]+[A-Z]*\b|\b[0-9]+[A-Z]+\b'),
            
            # 기술 키워드들
            'tech_keywords': re.compile(r'\b(?:API|SDK|JSON|XML|HTTP|HTTPS|URL|URI|JWT|OAuth|REST|SOAP|SSL|TLS|UTF|ASCII|Base64)\b', re.IGNORECASE),
            
            # 단어 경계에서 camelCase/PascalCase 추가 패턴
            'mixed_case': re.compile(r'\b[a-zA-Z]*[a-z][A-Z][a-zA-Z0-9]*\b'),
        }
        
        # 제외할 일반적인 단어들
        self.exclude_words = {
            'THE', 'AND', 'OR', 'BUT', 'FOR', 'WITH', 'FROM', 'TO', 'IN', 'ON', 'AT', 'BY',
            'AS', 'IS', 'ARE', 'WAS', 'WERE', 'BE', 'BEEN', 'HAVE', 'HAS', 'HAD', 'DO', 'DOES', 'DID',
            'WILL', 'WOULD', 'COULD', 'SHOULD', 'MAY', 'MIGHT', 'CAN', 'MUST', 'SHALL',
            'THIS', 'THAT', 'THESE', 'THOSE', 'A', 'AN', 'IF', 'WHEN', 'WHERE', 'HOW', 'WHY', 'WHAT',
            'WHO', 'WHICH', 'WHOSE', 'WHOM', 'ALL', 'ANY', 'SOME', 'MANY', 'MUCH', 'MORE', 'MOST',
            'OTHER', 'ANOTHER', 'EACH', 'EVERY', 'BOTH', 'EITHER', 'NEITHER', 'NOT', 'NO', 'YES'
        }
        
        # 한국어 제외 패턴 (한글이 포함된 것들은 제외)
        self.korean_pattern = re.compile(r'[가-힣]')
        
    def extract_tags(self, text: str, title: str = "") -> List[str]:
        """
        텍스트에서 기술 태그들을 추출합니다.
        
        Args:
            text: 분석할 텍스트
            title: 섹션 제목 (추가 태그 추출용)
            
        Returns:
            List[str]: 추출된 태그들
        """
        all_tags = set()
        
        # 제목에서도 태그 추출
        if title:
            title_tags = self._extract_from_text(title)
            all_tags.update(title_tags)
        
        # 본문에서 태그 추출
        content_tags = self._extract_from_text(text)
        all_tags.update(content_tags)
        
        # 빈도수 기반 필터링 및 정렬
        return self._filter_and_rank_tags(list(all_tags), text)
    
    def _extract_from_text(self, text: str) -> Set[str]:
        """텍스트에서 모든 패턴을 이용해 태그를 추출합니다."""
        tags = set()
        
        for pattern_name, pattern in self.patterns.items():
            matches = pattern.findall(text)
            
            for match in matches:
                # 그룹 매치 결과를 사용하는 패턴들
                if pattern_name in ['json_fields', 'table_codes', 'table_params']:
                    tag = match if isinstance(match, str) else match[0]
                else:
                    tag = match
                
                # 필터링 조건들
                if self._is_valid_tag(tag):
                    tags.add(tag)
        
        return tags
    
    def _is_valid_tag(self, tag: str) -> bool:
        """태그가 유효한지 검증합니다."""
        # 너무 짧거나 긴 것 제외 (긴 코드명도 허용)
        if len(tag) < 2 or len(tag) > 30:
            return False
            
        # 한글 포함된 것 제외
        if self.korean_pattern.search(tag):
            return False
            
        # 제외 단어 목록에 있는 것 제외 (단, 긴 코드는 예외)
        if len(tag) <= 5 and tag.upper() in self.exclude_words:
            return False
            
        # 숫자로만 이루어진 것 제외 (단, 의미있는 숫자 코드는 허용)
        if tag.isdigit() and len(tag) < 3:
            return False
            
        # 특수문자로만 이루어진 것 제외
        if re.match(r'^[^a-zA-Z0-9]+$', tag):
            return False
            
        # 점으로만 이루어진 것 제외
        if re.match(r'^\.+$', tag):
            return False
            
        # 대문자 코드는 더 관대하게 허용
        if re.match(r'^[A-Z][A-Z0-9_]*$', tag) and len(tag) >= 3:
            return True
            
        # camelCase 파라미터는 허용
        if re.match(r'^[a-z][a-zA-Z0-9]*[A-Z]', tag):
            return True
            
        return True
    
    def _filter_and_rank_tags(self, tags: List[str], text: str) -> List[str]:
        """태그들을 필터링하고 중요도에 따라 정렬합니다."""
        # 빈도수 계산
        tag_freq = Counter()
        for tag in tags:
            # 대소문자 구분 없이 빈도 계산
            occurrences = len(re.findall(re.escape(tag), text, re.IGNORECASE))
            tag_freq[tag] = occurrences
        
        # 중요도 계산 (길이, 빈도수, 패턴 유형 고려)
        scored_tags = []
        for tag, freq in tag_freq.items():
            score = self._calculate_tag_score(tag, freq, text)
            scored_tags.append((tag, score))
        
        # 점수 순으로 정렬하고 상위 태그들만 반환
        scored_tags.sort(key=lambda x: x[1], reverse=True)
        
        # 최대 30개까지만 반환, 최소 점수 0.5 이상인 것들만
        return [tag for tag, score in scored_tags[:30] if score >= 0.5]
    
    def _calculate_tag_score(self, tag: str, frequency: int, text: str) -> float:
        """태그의 중요도 점수를 계산합니다."""
        score = 0
        
        # 기본 빈도수 점수
        score += frequency * 0.5
        
        # 길이 점수 (적당한 길이가 좋음)
        if 3 <= len(tag) <= 20:  # 긴 코드명도 허용
            score += 2
        elif len(tag) >= 2:
            score += 1
            
        # 패턴별 가중치 (더 구체적으로)
        if re.match(r'^[A-Z]{6,}$', tag):  # 긴 대문자 코드 (CREDITCARD, TELCOMEMBERSHIP)
            score += 3.5
        elif re.match(r'^[A-Z]{3,}(?:[A-Z0-9_]*[A-Z0-9]+)+$', tag):  # 복합 코드 (SINGLE_PAYMENT_TRANSACTION)
            score += 4  # 가장 높은 점수
        elif re.match(r'^[a-z]+[A-Z][a-zA-Z0-9]*$', tag):  # camelCase (purchaseState, clientId)
            score += 3.5  # 높은 점수
        elif re.match(r'^[A-Z]{2,5}$', tag):  # 약어
            score += 3
        elif re.match(r'^[A-Z][A-Z0-9_]+$', tag):  # 일반 상수
            score += 2.5
        elif '_' in tag and tag.upper() == tag:  # CONSTANT_CASE
            score += 3
        elif re.match(r'^\d+\.\d+', tag):  # 버전 번호
            score += 2.5
        elif re.match(r'^[A-Z]*[0-9]+[A-Z]*$', tag):  # 숫자 포함 코드 (11PAY, V21)
            score += 2.5
        elif re.match(r'^[a-z]+_[a-z0-9_]+$', tag):  # snake_case
            score += 2
            
        # 기술 키워드 보너스
        tech_keywords = ['API', 'SDK', 'JSON', 'HTTP', 'URL', 'TOKEN', 'ID', 'KEY', 'PAY', 'CARD']
        if any(keyword in tag.upper() for keyword in tech_keywords):
            score += 1.5
            
        # 결제 관련 코드 보너스
        payment_keywords = ['PAY', 'CARD', 'CASH', 'POINT', 'COUPON', 'WALLET', 'BILL']
        if any(keyword in tag.upper() for keyword in payment_keywords):
            score += 1
            
        # 상태/환경 관련 코드 보너스
        status_keywords = ['COMPLETED', 'CANCELED', 'SANDBOX', 'COMMERCIAL', 'TEST']
        if any(keyword in tag.upper() for keyword in status_keywords):
            score += 1
            
        return score

class HierarchicalDocumentSplitter:
    """계층별 헤더 기반 문서 분할기"""
    
    def __init__(self, include_parent_context: bool = True, max_chunk_size: int = 2000, extract_tags: bool = True):
        """
        Args:
            include_parent_context: 상위 계층의 컨텍스트를 포함할지 여부
            max_chunk_size: 최대 청크 크기 (문자 수)
            extract_tags: 태그 추출 기능 사용 여부
        """
        self.include_parent_context = include_parent_context
        self.max_chunk_size = max_chunk_size
        self.extract_tags = extract_tags
        self.header_pattern = re.compile(r'^(#{1,4})\s+(.+?)(?:\s+<.*)?$', re.MULTILINE)
        
        # 태그 추출기 초기화
        if self.extract_tags:
            self.tag_extractor = TechnicalTermExtractor()
        
    def split_document(self, text: str) -> List[DocumentChunk]:
        """
        문서를 계층별로 분할합니다.
        
        Args:
            text: 분할할 마크다운 텍스트
            
        Returns:
            List[DocumentChunk]: 분할된 문서 청크들
        """
        lines = text.split('\n')
        chunks = []
        
        # 헤더 정보 추출
        headers = self._extract_headers(text)
        
        # 각 헤더별로 문서 청크 생성
        for i, header in enumerate(headers):
            start_line = header['line_num']
            end_line = headers[i + 1]['line_num'] - 1 if i + 1 < len(headers) else len(lines)
            
            # 해당 섹션의 내용 추출
            section_content = '\n'.join(lines[start_line:end_line])
            
            # 상위 컨텍스트 추가 (옵션)
            if self.include_parent_context:
                parent_context = self._get_parent_context(header, headers)
                if parent_context:
                    section_content = parent_context + '\n\n' + section_content
            
            # 태그 추출
            tags = []
            if self.extract_tags:
                tags = self.tag_extractor.extract_tags(section_content, header['title'])
            
            # 청크 생성
            chunk = DocumentChunk(
                content=section_content,
                level=header['level'],
                title=header['title'],
                full_path=header['path'],
                metadata={
                    'level': header['level'],
                    'section_id': f"section_{i}",
                    'parent_titles': header['path'][:-1],
                    'char_count': len(section_content),
                    'line_range': f"{start_line}-{end_line}",
                    'tags': tags
                },
                start_line=start_line,
                end_line=end_line,
                tags=tags
            )
            
            chunks.append(chunk)
            
            # 하위 레벨별로도 청크 생성 (계층적 접근)
            sub_chunks = self._create_hierarchical_chunks(header, section_content, start_line)
            chunks.extend(sub_chunks)
        
        return chunks
    
    def _extract_headers(self, text: str) -> List[Dict]:
        """텍스트에서 헤더 정보를 추출합니다."""
        lines = text.split('\n')
        headers = []
        path_stack: List[str] = []
        
        for line_num, line in enumerate(lines):
            match = self.header_pattern.match(line)
            if match:
                level = len(match.group(1))  # # 개수
                title = match.group(2).strip()
                
                # 경로 스택 관리
                while len(path_stack) >= level:
                    path_stack.pop()
                
                path_stack.append(title)
                
                headers.append({
                    'level': level,
                    'title': title,
                    'path': path_stack.copy(),
                    'line_num': line_num
                })
        
        return headers
    
    def _get_parent_context(self, current_header: Dict, all_headers: List[Dict]) -> str:
        """현재 헤더의 상위 컨텍스트를 가져옵니다."""
        parent_context = []
        
        # 상위 레벨 헤더들의 제목을 컨텍스트로 추가
        for parent_title in current_header['path'][:-1]:
            parent_context.append(f"상위 섹션: {parent_title}")
        
        return '\n'.join(parent_context) if parent_context else ""
    
    def _create_hierarchical_chunks(self, header: Dict, content: str, start_line: int) -> List[DocumentChunk]:
        """계층적으로 청크를 생성합니다."""
        chunks = []
        
        # 긴 내용을 더 작은 청크로 분할
        if len(content) > self.max_chunk_size:
            sub_chunks = self._split_long_content(content, header, start_line)
            chunks.extend(sub_chunks)
        
        return chunks
    
    def _split_long_content(self, content: str, header: Dict, start_line: int) -> List[DocumentChunk]:
        """긴 내용을 작은 청크로 분할합니다."""
        chunks = []
        paragraphs = content.split('\n\n')
        current_chunk = ""
        chunk_count = 0
        
        for para in paragraphs:
            if len(current_chunk + para) > self.max_chunk_size and current_chunk:
                # 태그 추출
                tags = []
                if hasattr(self, 'tag_extractor') and self.extract_tags:
                    tags = self.tag_extractor.extract_tags(current_chunk.strip(), f"{header['title']} (Part {chunk_count + 1})")
                
                # 현재 청크를 저장
                chunk = DocumentChunk(
                    content=current_chunk.strip(),
                    level=header['level'] + 1,  # 하위 레벨로 설정
                    title=f"{header['title']} (Part {chunk_count + 1})",
                    full_path=header['path'] + [f"Part {chunk_count + 1}"],
                    metadata={
                        'level': header['level'] + 1,
                        'section_id': f"section_{header['title']}_{chunk_count}",
                        'parent_titles': header['path'],
                        'char_count': len(current_chunk),
                        'is_sub_chunk': True,
                        'tags': tags
                    },
                    start_line=start_line,
                    end_line=start_line,
                    tags=tags
                )
                chunks.append(chunk)
                current_chunk = para
                chunk_count += 1
            else:
                current_chunk += "\n\n" + para if current_chunk else para
        
        # 마지막 청크 추가
        if current_chunk.strip():
            # 태그 추출
            tags = []
            if hasattr(self, 'tag_extractor') and self.extract_tags:
                tags = self.tag_extractor.extract_tags(current_chunk.strip(), f"{header['title']} (Part {chunk_count + 1})")
            
            chunk = DocumentChunk(
                content=current_chunk.strip(),
                level=header['level'] + 1,
                title=f"{header['title']} (Part {chunk_count + 1})",
                full_path=header['path'] + [f"Part {chunk_count + 1}"],
                metadata={
                    'level': header['level'] + 1,
                    'section_id': f"section_{header['title']}_{chunk_count}",
                    'parent_titles': header['path'],
                    'char_count': len(current_chunk),
                    'is_sub_chunk': True,
                    'tags': tags
                },
                start_line=start_line,
                end_line=start_line,
                tags=tags
            )
            chunks.append(chunk)
        
        return chunks
    
    def get_chunks_by_level(self, chunks: List[DocumentChunk], level: int) -> List[DocumentChunk]:
        """특정 레벨의 청크들만 반환합니다."""
        return [chunk for chunk in chunks if chunk.level == level]
    
    def find_relevant_chunks(self, chunks: List[DocumentChunk], query: str) -> List[DocumentChunk]:
        """쿼리와 관련된 청크들을 찾습니다."""
        relevant_chunks = []
        query_lower = query.lower()
        
        for chunk in chunks:
            # 제목이나 내용에 쿼리 키워드가 포함된 청크 찾기
            if (query_lower in chunk.title.lower() or 
                query_lower in chunk.content.lower()):
                relevant_chunks.append(chunk)
        
        return relevant_chunks
    
    def find_chunks_by_tags(self, chunks: List[DocumentChunk], target_tags: List[str]) -> List[DocumentChunk]:
        """특정 태그들을 포함하는 청크들을 찾습니다."""
        relevant_chunks = []
        target_tags_lower = [tag.lower() for tag in target_tags]
        
        for chunk in chunks:
            chunk_tags_lower = [tag.lower() for tag in chunk.tags]
            # 타겟 태그 중 하나라도 포함하면 관련 청크로 판정
            if any(target_tag in chunk_tags_lower for target_tag in target_tags_lower):
                relevant_chunks.append(chunk)
        
        return relevant_chunks
    
    def get_all_tags(self, chunks: List[DocumentChunk]) -> Dict[str, int]:
        """모든 청크에서 태그들과 빈도수를 수집합니다."""
        tag_counter = Counter()
        
        for chunk in chunks:
            for tag in chunk.tags:
                tag_counter[tag] += 1
        
        return dict(tag_counter)

In [3]:

# 개선된 태그 추출 기능 테스트
print("=== 개선된 태그 추출 기능 테스트 ===\n")

# 간단한 API 파라미터 테스트
test_params = "purchaseState clientId productId msgVersion purchaseTimeMillis priceCurrencyCode billingKey purchaseToken"
print("테스트 파라미터:", test_params)

# 태그 추출기 테스트
tag_extractor = TechnicalTermExtractor()
simple_tags = tag_extractor.extract_tags(test_params, "API Parameters")

print("\n간단한 camelCase 파라미터 추출 결과:")
for i, tag in enumerate(simple_tags, 1):
    print(f"{i:2d}. {tag}")

# PNS 문서 샘플로 완전한 테스트
pns_sample_text = """
| Element Name       | Data Type     | Description                                                                   |
| ------------------ | ------------- | ----------------------------------------------------------------------------- |
| msgVersion         | String        | 메시지 버전 (개발: 3.1.0D, 상용: 3.1.0)                                         |
| clientId           | String        | 앱의 클라이언트 ID                                                              |
| productId          | String        | 인앱상품의 상품 ID                                                              |
| messageType        | String        | SINGLE_PAYMENT_TRANSACTION 고정                                               |
| purchaseId         | String        | 구매 ID                                                                        |
| developerPayload   | String        | 구매건을 식별하기 위해 개발사에서 관리하는 식별자                                   |
| purchaseTimeMillis | Long          | 원스토어 결제 시스템에서 결제가 완료된 시간(ms)                                     |
| purchaseState      | String        | COMPLETED : 결제완료 / CANCELED : 취소                                          |
| price              | String        | 결제 금액                                                                       |
| priceCurrencyCode  | String        | 결제 금액 통화코드(KRW, USD, ...)                                               |
| billingKey         | String        | 확장 기능용 결제 키                                                              |
| purchaseToken      | String        | 구매토큰                                                                        |
| environment        | String        | 결제환경 (개발: SANDBOX, 상용: COMMERCIAL)                                       |

{
    "msgVersion" : "3.1.0",
    "clientId":"0000000001",
    "productId":"0900001234",
    "messageType":"SINGLE_PAYMENT_TRANSACTION",
    "purchaseId":"SANDBOX3000000004564",
    "developerPayload":"OS_000211234",
    "purchaseTimeMillis":24431212233,
    "purchaseState":"COMPLETED",
    "price":"10000",
    "priceCurrencyCode":"KRW",
    "billingKey" : "TOKEN...",
    "purchaseToken" : "TOKEN...",
    "environment" : "SANDBOX"
}
"""

print("\n" + "="*50)
print("PNS 문서 샘플에서 태그 추출:")

extracted_tags = tag_extractor.extract_tags(pns_sample_text, "PNS Payment Notification Service")

print("\n추출된 태그들:")
for i, tag in enumerate(extracted_tags, 1):
    print(f"{i:2d}. {tag}")

print(f"\n총 {len(extracted_tags)}개의 태그가 추출되었습니다.")

# 중요한 camelCase 태그 확인
important_camel_tags = ['purchaseState', 'clientId', 'productId', 'msgVersion', 
                        'purchaseTimeMillis', 'priceCurrencyCode', 'developerPayload', 
                        'billingKey', 'purchaseToken']

print("\n=== 중요한 camelCase 태그 확인 ===")
for tag in important_camel_tags:
    status = "✅ 추출됨" if tag in extracted_tags else "❌ 누락됨"
    print(f"{tag}: {status}")

print("\n" + "="*50)

=== 개선된 태그 추출 기능 테스트 ===

테스트 파라미터: purchaseState clientId productId msgVersion purchaseTimeMillis priceCurrencyCode billingKey purchaseToken

간단한 camelCase 파라미터 추출 결과:
 1. billingKey
 2. clientId
 3. purchaseToken
 4. productId
 5. API
 6. msgVersion
 7. priceCurrencyCode
 8. purchaseTimeMillis
 9. purchaseState

PNS 문서 샘플에서 태그 추출:

추출된 태그들:
 1. ID
 2. TOKEN
 3. developerPayload
 4. billingKey
 5. SINGLE_PAYMENT_TRANSACTION
 6. clientId
 7. purchaseToken
 8. productId
 9. purchaseId
10. SANDBOX
11. COMPLETED
12. SANDBOX3000000004564
13. CANCELED
14. COMMERCIAL
15. messageType
16. purchaseState
17. purchaseTimeMillis
18. msgVersion
19. priceCurrencyCode
20. KRW
21. 3.1.0
22. USD
23. 3.1.0D
24. 24431212233
25. 0000000001
26. OS_000211234
27. 0900001234
28. PNS
29. 10000
30. price

총 30개의 태그가 추출되었습니다.

=== 중요한 camelCase 태그 확인 ===
purchaseState: ✅ 추출됨
clientId: ✅ 추출됨
productId: ✅ 추출됨
msgVersion: ✅ 추출됨
purchaseTimeMillis: ✅ 추출됨
priceCurrencyCode: ✅ 추출됨
developerPayload: ✅ 추출됨
billingKey: ✅

In [4]:
# HierarchicalDocumentSplitter에서 태그 추출 기능 테스트
print("\n=== HierarchicalDocumentSplitter 통합 테스트 ===\n")

# 태그 추출 기능이 포함된 분할기 생성
splitter_with_tags = HierarchicalDocumentSplitter(
    include_parent_context=False,  # 상위 컨텍스트는 테스트를 위해 끔
    max_chunk_size=1000,  # 작은 청크 크기로 테스트
    extract_tags=True  # 태그 추출 기능 활성화
)

# PNS 샘플 텍스트로 문서 분할 및 태그 추출
chunks = splitter_with_tags.split_document(pns_sample_text)

print(f"총 {len(chunks)}개의 청크가 생성되었습니다.\n")

# 각 청크별로 태그 정보 출력
for i, chunk in enumerate(chunks, 1):
    print(f"=== 청크 {i} ===")
    print(f"제목: {chunk.title}")
    print(f"레벨: {chunk.level}")
    print(f"경로: {' > '.join(chunk.full_path)}")
    print(f"내용 길이: {len(chunk.content)} 문자")
    print(f"태그 개수: {len(chunk.tags)}")
    
    if chunk.tags:
        print("태그:", ", ".join(chunk.tags))
    else:
        print("태그: 없음")
    
    print(f"내용 미리보기: {chunk.content[:150]}...")
    print("-" * 80)
    print()



=== HierarchicalDocumentSplitter 통합 테스트 ===

총 0개의 청크가 생성되었습니다.



In [6]:
# 원본 문서 파일 경로 설정
document_path = "data/dev_center_guide_allmd_touched.md"

# 문서 읽기
with open(document_path, 'r', encoding='utf-8') as f:
    document_text = f.read()

# PNS 섹션만 추출 (라인 5073부터 약 200라인)
lines = document_text.split('\n')
pns_section = '\n'.join(lines[5072:5272]) 

print(pns_section)

# 07. PNS(Payment Notification Service) 이용하기

## **개요**  

PNS는 Payment Notification Service의 약자입니다. PNS는 모바일의 네트워크 연결 불안정성을 보완하기 위해 개발사가 지정한 서버로 원스토어의 서버가 개별 사용자의 결제 상태(결제 완료, 결제 취소)를 메시지로 전송하여 결제 트랜젝션의 상태를 손실없이 알려주기 위한 용도의 기능입니다. 즉, 개발사가 지정한 서버에서 원스토어가 정의한 규칙에 맞추어 API를 구현하면 해당 API를 원스토어의 결제 담당 서버에서 호출하는 형태입니다.

Server to Server, 즉 서버간에 데이터를 전송한다고 할지라도 네트워크 문제로 메세지 전송 실패가 발생하기 때문에 200 OK로 응답을 인지하지 못할 경우 반복하여 메시지가 전송될 수 있습니다. 개발사의 서버는 메시지를 수신후 정의된 응답을 하여야 원스토어는 개발사 서버가 정상적으로 메시지를 전달 받았음을 인지합니다.

결제 트렌젝션의 유형에 따라 아래와 같은 유형의 메세지가 전송됩니다.

* 인앱상품 결제 또는 결제취소가 발생하면 원스토어가 개발사 서버로 알림을 전송하는 PNS(Payment Notification Service)&#x20;
* 구독 상태가 변경되면 개발사 서버로 알림을 전송하는 SNS (Subscription Notifacation Service)&#x20;

{% hint style="warning" %}
Notification은 발송/수신 서버의 상태에 따라 지연 또는 유실될 수 있으므로, notification 수신을 기준으로 상품(서비스)을 제공하는 것은 권장하지 않습니다.&#x20;

정상적인 결제 건인지 Server to Server로 확인하기를 원하신다면 PNS notification을 이용하는 대신, 관련 서버 API로 조회하는 것을 권장합니다.

원스토어는 검증 및 모니터링 목적으로 결제 테스트를 진행할 수 있으며, 해당 테스트 건들도 결제/결제취소 시 동일하게

In [7]:
# 실제 문서 파일에서 PNS 섹션 태그 추출 테스트
print("\n=== 실제 문서 파일 테스트 ===\n")

# 원본 문서 파일 경로 설정
document_path = "data/dev_center_guide_allmd_touched.md"

# 문서 읽기
with open(document_path, 'r', encoding='utf-8') as f:
    document_text = f.read()

# PNS 섹션만 추출 (라인 5073부터 약 200라인)
lines = document_text.split('\n')
pns_section = '\n'.join(lines[5072:5272])  # 5073라인부터 200라인

print("PNS 섹션에서 추출된 태그들:")
print("-" * 40)

# PNS 섹션에서 태그 추출
pns_tags = tag_extractor.extract_tags(pns_section, "PNS 이용하기")
for i, tag in enumerate(pns_tags, 1):
    print(f"{i:2d}. {tag}")

print(f"\n총 {len(pns_tags)}개의 태그가 추출되었습니다.")

# 태그 카테고리별 분류
categories = {
    'API_파라미터': [],
    '상수값': [],
    '약어': [],
    '버전': [],
    '기술용어': []
}

for tag in pns_tags:
    if re.match(r'^[a-z][a-zA-Z0-9]*[A-Z]', tag):  # camelCase
        categories['API_파라미터'].append(tag)
    elif re.match(r'^[A-Z][A-Z0-9_]+$', tag):  # CONSTANT_CASE
        categories['상수값'].append(tag)
    elif re.match(r'^[A-Z]{2,5}$', tag):  # 약어
        categories['약어'].append(tag)
    elif re.match(r'^\d+\.\d+', tag):  # 버전
        categories['버전'].append(tag)
    else:
        categories['기술용어'].append(tag)

print("\n=== 태그 카테고리별 분류 ===")
for category, tags in categories.items():
    if tags:
        print(f"\n{category}: {', '.join(tags)}")



=== 실제 문서 파일 테스트 ===

PNS 섹션에서 추출된 태그들:
----------------------------------------
 1. SIGNATURE
 2. KEY
 3. String
 4. signature
 5. ID
 6. JSON
 7. paymentMethod
 8. PUBLIC
 9. publicKey
10. PNS
11. paymentTypeList
12. developerPayload
13. URL
14. billingKey
15. SANDBOX
16. TOKEN
17. ONE
18. clientId
19. NAVERPAY
20. PAYPAL
21. productId
22. json
23. purchaseId
24. CREDITCARD
25. ONEPAY
26. SINGLE_PAYMENT_TRANSACTION
27. MYCARD
28. ONESTORECASH
29. isTestMdn
30. 11PAY

총 30개의 태그가 추출되었습니다.

=== 태그 카테고리별 분류 ===

API_파라미터: paymentMethod, publicKey, paymentTypeList, developerPayload, billingKey, clientId, productId, purchaseId, isTestMdn

상수값: SIGNATURE, KEY, ID, JSON, PUBLIC, PNS, URL, SANDBOX, TOKEN, ONE, NAVERPAY, PAYPAL, CREDITCARD, ONEPAY, SINGLE_PAYMENT_TRANSACTION, MYCARD, ONESTORECASH

기술용어: String, signature, json, 11PAY


In [8]:
# 원본 문서 파일 경로 설정
document_path = "data/dev_center_guide_allmd_touched.md"

# 문서 읽기
with open(document_path, 'r', encoding='utf-8') as f:
    document_text = f.read()

print(f"문서 로드 완료: {len(document_text):,}자")
print(f"총 라인 수: {len(document_text.split())}")

splitter = HierarchicalDocumentSplitter(
    include_parent_context=True,  # 상위 컨텍스트 포함
    max_chunk_size=2000  # 최대 청크 크기
)

chunks = splitter.split_document(document_text)

cnt = 0

for chunk in chunks:
  if('PNS' in chunk.title or 'PNS' in chunk.content):
    print(f"idx: {cnt}")
    print(f"title: {chunk.title}")
    print(f"content: {chunk.content}")
    print(f"full_path: {chunk.full_path}")
    print(f"level: {chunk.level}")
    print(f"start_line: {chunk.start_line}")
    print(f"end_line: {chunk.end_line}")
    print(f"metadata: {chunk.metadata}")
    print("-" * 100)
    cnt += 1

print(f"PNS 관련 청크 수: {cnt}")

문서 로드 완료: 383,563자
총 라인 수: 35227
idx: 0
title: 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드
content: # 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드

원스토어의 최신 인앱결제 API V7(SDK V21)이 출시되었습니다.

보다 강력하고 다양한 기능을 지원하는 최신 버전을 적용해보세요.

{% hint style="info" %}
API V4(SDK V16) 이하 버전과는 호환되지 않습니다. 인앱결제 API V4(SDK V16)에 대한 안내 및 다운로드는 [여기](old-version/v16)를 클릭해주세요.
{% endhint %}

{% hint style="info" %}
현재 판매중인 앱을 대한민국 외 국가/지역으로 배포하기 위해서는 아래 가이드를 참고해주세요

* [대한민국 외 국가 및 지역 배포를 위한 가이드](../glb)
{% endhint %}

If you are comfortable with English, please change the language to English from the upper left side in this page.

* [01. 원스토어 인앱결제 개요](v21/ov)
* [02. 원스토어 인앱결제 적용을 위한 사전준비](v21/pre)
* [03. 결제 테스트 및 보안](v21/test)
* [04. 원스토어 인앱결제 SDK를 사용해 구현하기](v21/sdk)
* [05. 원스토어 인앱결제 레퍼런스](v21/references)
* [06. 원스토어 인앱결제 서버 API (API V7)](v21/serverapi)
* [07. PNS(Payment Notification Service) 이용하기](v21/pns)
* [08. 정기 결제 적용하기](v21/subs)
* [09. 원스토어 인앱결제 릴리즈 노트](v21/releasenote)
* [10. Sample App Download](v21/sample)
* [11.

In [9]:
from langchain.docstore.document import Document

# @dataclass
# class DocumentChunk:
#     """문서 청크를 나타내는 클래스"""
#     content: str
#     level: int  # 헤더 레벨 (1=H1, 2=H2, 3=H3, 4=H4)
#     title: str
#     full_path: List[str]  # 계층적 경로 (예: ["07. PNS", "PNS 상세", "Payment Notification"])
#     metadata: Dict
#     start_line: int
#     end_line: int
#     tags: List[str]  # 문서에서 추출된 태그들 (특수용어, 코드값, API 파라미터 등)

docs = []
for chunk in chunks:
  doc = Document(
    page_content=f"[title]: {chunk.title}\n[section_path]: {chunk.full_path}\n[tags]: {chunk.tags}\n\n[contents]:\n{chunk.content}",
    metadata=chunk.metadata
  )
  docs.append(doc)

for doc in docs:
  print(doc.page_content)
  # print(doc.metadata)
  print("-" * 100)

# chunk = DocumentChunk(
#                 content=current_chunk.strip(),
#                 level=header['level'] + 1,
#                 title=f"{header['title']} (Part {chunk_count + 1})",
#                 full_path=header['path'] + [f"Part {chunk_count + 1}"],
#                 metadata={
#                     'level': header['level'] + 1,
#                     'section_id': f"section_{header['title']}_{chunk_count}",
#                     'parent_titles': header['path'],
#                     'char_count': len(current_chunk),
#                     'is_sub_chunk': True,
#                     'tags': tags
#                 },
#                 start_line=start_line,
#                 end_line=start_line,
#                 tags=tags
#             )



[title]: 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드
[section_path]: ['원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드']
[tags]: ['V21', 'SDK', 'API', 'sdk', 'PNS', 'V16', 'IAP', 'V7', 'V4', 'info']

[contents]:
# 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드

원스토어의 최신 인앱결제 API V7(SDK V21)이 출시되었습니다.

보다 강력하고 다양한 기능을 지원하는 최신 버전을 적용해보세요.

{% hint style="info" %}
API V4(SDK V16) 이하 버전과는 호환되지 않습니다. 인앱결제 API V4(SDK V16)에 대한 안내 및 다운로드는 [여기](old-version/v16)를 클릭해주세요.
{% endhint %}

{% hint style="info" %}
현재 판매중인 앱을 대한민국 외 국가/지역으로 배포하기 위해서는 아래 가이드를 참고해주세요

* [대한민국 외 국가 및 지역 배포를 위한 가이드](../glb)
{% endhint %}

If you are comfortable with English, please change the language to English from the upper left side in this page.

* [01. 원스토어 인앱결제 개요](v21/ov)
* [02. 원스토어 인앱결제 적용을 위한 사전준비](v21/pre)
* [03. 결제 테스트 및 보안](v21/test)
* [04. 원스토어 인앱결제 SDK를 사용해 구현하기](v21/sdk)
* [05. 원스토어 인앱결제 레퍼런스](v21/references)
* [06. 원스토어 인앱결제 서버 API (API V7)](v21/serverapi)
* [07. PNS(Payment Notification Service) 이용하기](v21/pns)
* [08. 정기 결제 적

In [11]:
fixed_model_name = "bge-m3:latest"
model_file_name = "models/faiss_rag_base_bge-m3"

def save_documents(docs: List[Document], output_path: str):
    """
    List[Document]를 pickle 파일로 저장합니다.
    
    Args:
        docs (List[Document]): 저장할 문서 리스트
        output_path (str): 저장할 디렉토리 경로
    """
    os.makedirs(output_path, exist_ok=True)
    docs_file_path = os.path.join(output_path, "documents.pkl")
    
    with open(docs_file_path, "wb") as f:
        pickle.dump(docs, f)
    
    print(f"✅ 문서 저장 완료: {docs_file_path}")
    print(f"📄 저장된 문서 수: {len(docs)}")
    
def embed_and_save_with_docs(docs: List[Document], output_path: str, model_name: str = "bge-m3:latest"):
    """
    문서를 임베딩하고 FAISS 데이터베이스로 저장하며, 동시에 원본 문서도 저장합니다.
    
    Args:
        docs (List[Document]): 처리할 문서 리스트
        output_path (str): 저장할 디렉토리 경로
        model_name (str): 임베딩 모델명
    """
    # 임베딩 모델 초기화
    embedding_model = OllamaEmbeddings(model=model_name)
    
    # FAISS 데이터베이스 생성 및 저장
    db = FAISS.from_documents(docs, embedding_model)
    db.save_local(output_path)
    print(f"✅ 임베딩 저장 완료: {output_path}")
    
    # 원본 문서도 함께 저장
    save_documents(docs, output_path)
    
# embed_and_save_with_docs(docs, "models/faiss_rag_base_bge-m3")

In [14]:
# 개선된 메타데이터 기반 검색기 구현
class MetadataAwareRetriever:
    """메타데이터를 활용한 스마트 검색기"""
    
    def __init__(self, ensemble_retriever):
        self.ensemble_retriever = ensemble_retriever
        
    def smart_retrieve(self, query: str, k: int = 10):
        """메타데이터 기반 스마트 검색"""
        # 1. 기본 앙상블 검색으로 더 많은 후보 확보
        candidates = self.ensemble_retriever.invoke(query)
        
        # 2. 쿼리에서 키워드 추출
        query_keywords = self._extract_query_keywords(query)
        
        # 3. 메타데이터 기반 점수 계산
        scored_docs = []
        for doc in candidates:
            score = self._calculate_comprehensive_score(doc, query, query_keywords)
            scored_docs.append((score, doc))
        
        # 4. 점수순 정렬 후 상위 k개 반환
        scored_docs.sort(key=lambda x: x[0], reverse=True)
        return [doc for score, doc in scored_docs[:k]]
    
    def _extract_query_keywords(self, query: str):
        """쿼리에서 도메인별 키워드 추출"""
        keywords = {
            'modules': [],      # 모듈명 (PNS, IAP, SDK 등)
            'entities': [],     # 엔티티명 (purchaseState, clientId 등)
            'general': []       # 일반 키워드
        }
        
        # 모듈명 패턴
        module_patterns = [
            r'\bPNS\b', r'\bIAP\b', r'\bSDK\b', r'\bAPI\b',
            r'Payment Notification', r'인앱결제'
        ]
        
        # 엔티티명 패턴 (camelCase, 주요 필드명)
        entity_patterns = [
            r'\b[a-z]+[A-Z][a-zA-Z0-9]*\b',  # camelCase
            r'\b[A-Z][A-Z0-9_]+\b'           # CONSTANT_CASE
        ]
        
        # 모듈명 추출
        for pattern in module_patterns:
            matches = re.findall(pattern, query, re.IGNORECASE)
            keywords['modules'].extend(matches)
        
        # 엔티티명 추출
        for pattern in entity_patterns:
            matches = re.findall(pattern, query)
            keywords['entities'].extend(matches)
        
        # 일반 키워드
        words = query.split()
        keywords['general'] = [word.strip('?.,!') for word in words if len(word) > 2]
        
        return keywords
    
    def _calculate_comprehensive_score(self, doc, query: str, keywords):
        """종합적인 점수 계산"""
        score = 1.0  # 기본 점수
        
        # 1. 모듈 일치 점수 (가장 중요!)
        module_score = self._calculate_module_score(doc, keywords['modules'])
        score += module_score * 5.0  # 높은 가중치
        
        # 2. 섹션 경로 일치 점수
        section_score = self._calculate_section_score(doc, keywords['modules'])
        score += section_score * 3.0
        
        # 3. 태그 일치 점수
        tag_score = self._calculate_tag_score(doc, keywords['entities'])
        score += tag_score * 2.0
        
        # 4. 복합 쿼리 보너스
        if self._is_compound_query_match(doc, query):
            score += 3.0
        
        return score
    
    def _calculate_module_score(self, doc, modules):
        """모듈명 기반 점수 계산"""
        score = 0.0
        
        # section_path에서 모듈 확인
        if hasattr(doc, 'metadata') and 'section_path' in doc.metadata:
            section_path_str = ' '.join(doc.metadata['section_path']).lower()
            
            for module in modules:
                if module.lower() in section_path_str:
                    score += 2.0  # 섹션 경로에 모듈명이 있으면 높은 점수
        
        return score
    
    def _calculate_section_score(self, doc, modules):
        """섹션 경로 기반 점수"""
        score = 0.0
        
        if hasattr(doc, 'metadata') and 'section_path' in doc.metadata:
            section_path = doc.metadata['section_path']
            
            # 섹션 깊이가 깊을수록 구체적 (보너스)
            score += len(section_path) * 0.1
            
            # 모듈 관련 섹션인지 확인
            for section in section_path:
                if any(module.lower() in section.lower() for module in modules):
                    score += 1.0
        
        return score
    
    def _calculate_tag_score(self, doc, entities):
        """태그 기반 점수"""
        score = 0.0
        
        if hasattr(doc, 'metadata') and 'tags' in doc.metadata:
            tags = doc.metadata['tags']
            
            for entity in entities:
                if entity in tags:
                    score += 2.0  # 정확한 태그 매칭
                else:
                    # 부분 매칭
                    for tag in tags:
                        if entity.lower() in tag.lower():
                            score += 0.5
        
        return score
    
    def _is_compound_query_match(self, doc, query: str):
        """복합 쿼리 매칭 확인 (예: "PNS의 purchaseState")"""
        query_lower = query.lower()
        
        # "A의 B" 패턴 감지
        if '의' in query or 'of' in query_lower:
            # PNS와 purchaseState가 모두 문서에 있는지 확인
            if 'pns' in query_lower and 'purchasestate' in query_lower:
                section_path_str = ''
                if hasattr(doc, 'metadata') and 'section_path' in doc.metadata:
                    section_path_str = ' '.join(doc.metadata['section_path']).lower()
                
                tags_str = ''
                if hasattr(doc, 'metadata') and 'tags' in doc.metadata:
                    tags_str = ' '.join(doc.metadata['tags']).lower()
                
                content = doc.page_content.lower()
                
                # PNS가 섹션 경로에 있고, purchaseState가 태그나 내용에 있으면 복합 매칭
                has_pns = 'pns' in section_path_str or 'payment notification' in section_path_str
                has_purchase_state = 'purchasestate' in tags_str or 'purchasestate' in content
                
                return has_pns and has_purchase_state
        
        return False

print("✅ 메타데이터 기반 스마트 검색기 정의 완료")


✅ 메타데이터 기반 스마트 검색기 정의 완료


In [13]:
###### Retriever Check from FAISS Vector DB ######

from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.documents import Document
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers import BM25Retriever


# ✅ 3. 임베딩 모델 초기화 (Ollama)
embedding_model = OllamaEmbeddings(model=fixed_model_name)

# 저장된 데이터를 로드
loaded_db = FAISS.load_local(
    folder_path=model_file_name,
    # index_name="index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True,
)

retriever = loaded_db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 30, "fetch_k": 70, "lambda_mult": 0.7}
    # search_type="similarity",
    # search_kwargs={"k": 50}
)

bm25 = BM25Retriever.from_documents(
    docs,
    bm25_params={"k1": 1.5, "b": 0.75}
)

bm25.k = 20

ensembled_retriever = EnsembleRetriever(
    retrievers=[bm25, retriever],
    weights=[0.5, 0.5]
)


res = ensembled_retriever.invoke(
    "PNS의 purchaseState 값은 무엇인가요?"
)

# res = retriever.invoke(
#     "원스토어 인앱결제의 PNS의 개념을 설명해주세요"
# )

print(f"검색된 문서 수: {len(res)}")

idx = 0
for doc in res: 
    print(f"--- doc_index: {idx} ---")
    print(doc.page_content)  # Print first 100 characters of each document
    # print(doc.metadata)
    # print('-' * 40)
    idx += 1

검색된 문서 수: 50
--- doc_index: 0 ---
[title]: **OAuth API 상세**
[section_path]: ['06. 원스토어 인앱결제 서버 API (API V7)', '**원스토어 OAuth**', '**OAuth API 상세**']
[tags]: ['API', 'api', 'V7', 'OAuth']

[contents]:
상위 섹션: 06. 원스토어 인앱결제 서버 API (API V7)
상위 섹션: **원스토어 OAuth**

### **OAuth API 상세** <a href="#id-06.-api-apiv7-oauthapi" id="id-06.-api-apiv7-oauthapi"></a>

**client\_id 및 client\_secret 확인**

Client\_id와 Client\_secret 값은 "라이선스 관리" 메뉴에서 확인할 수 있습니다.


--- doc_index: 1 ---
[title]: getPurchaseState
[section_path]: ['PurchaseData', 'Public methods', 'getPurchaseState']
[tags]: ['getPurchaseState', 'PurchaseState', 'PurchaseData', 'int']

[contents]:
상위 섹션: PurchaseData
상위 섹션: Public methods

### getPurchaseState <a href="#id-c-purchasedata-getpurchasestate" id="id-c-purchasedata-getpurchasestate"></a>

```
int getPurchaseState()
```

구매 상태를 나타내는 값으로 \[A]PurchaseData.PurchaseState 중 하나를 반환합니다.

| **Returns:** |             |
| ------------ | ----------- |
| int          | <p><br></p> |
--- doc_i

In [15]:
# 개선된 메타데이터 기반 검색기로 테스트
smart_retriever = MetadataAwareRetriever(ensembled_retriever)

print("=== 기존 검색 결과 ===")
basic_results = ensembled_retriever.invoke("PNS의 purchaseState 값은 무엇인가요?")
print(f"기존 검색된 문서 수: {len(basic_results)}")

# 정답 문서 위치 찾기
target_found_at = -1
for i, doc in enumerate(basic_results):
    if hasattr(doc, 'metadata') and 'section_path' in doc.metadata:
        if any('PNS' in section for section in doc.metadata['section_path']):
            if hasattr(doc, 'metadata') and 'tags' in doc.metadata:
                if 'purchaseState' in doc.metadata['tags']:
                    target_found_at = i + 1
                    break

print(f"기존 방식에서 정답 문서 순위: {target_found_at}위")

print("\n" + "="*50)
print("=== 개선된 검색 결과 ===")
smart_results = smart_retriever.smart_retrieve("PNS의 purchaseState 값은 무엇인가요?", k=10)
print(f"개선된 검색된 문서 수: {len(smart_results)}")

# 개선된 방식에서 정답 문서 위치
smart_target_found_at = -1
for i, doc in enumerate(smart_results):
    if hasattr(doc, 'metadata') and 'section_path' in doc.metadata:
        if any('PNS' in section for section in doc.metadata['section_path']):
            if hasattr(doc, 'metadata') and 'tags' in doc.metadata:
                if 'purchaseState' in doc.metadata['tags']:
                    smart_target_found_at = i + 1
                    break

print(f"개선된 방식에서 정답 문서 순위: {smart_target_found_at}위")

print("\n=== 상위 3개 결과 비교 ===")
for i, doc in enumerate(smart_results[:3], 1):
    print(f"--- 순위 {i} ---")
    if hasattr(doc, 'metadata'):
        if 'section_path' in doc.metadata:
            print(f"섹션 경로: {' > '.join(doc.metadata['section_path'])}")
        if 'tags' in doc.metadata:
            relevant_tags = [tag for tag in doc.metadata['tags'] if any(keyword in tag.lower() for keyword in ['purchase', 'pns', 'state', 'complete'])]
            print(f"관련 태그: {relevant_tags}")
    print(f"내용: {doc.page_content[:150]}...")
    print()


=== 기존 검색 결과 ===
기존 검색된 문서 수: 50
기존 방식에서 정답 문서 순위: -1위

=== 개선된 검색 결과 ===
개선된 검색된 문서 수: 10
개선된 방식에서 정답 문서 순위: -1위

=== 상위 3개 결과 비교 ===
--- 순위 1 ---
관련 태그: ['purchaseId', 'acknowledgeState', 'purchaseState', 'purchaseTime', 'consumptionState', 'getPurchaseDetails']
내용: [title]: getPurchaseDetails (구매상품 상세조회) (Part 2)
[section_path]: ['06. 원스토어 인앱결제 서버 API (API V7)', '**서버 API 상세**', 'getPurchaseDetails (구매상품 상세조회)', ...

--- 순위 2 ---
관련 태그: ['purchaseId', 'purchaseToken', 'lastPurchaseId', 'acknowledgeState', 'purchaseState', 'lastPurchaseState']
내용: [title]: &#x20;Server API 변경 (Part 2)
[section_path]: ['09. 원스토어 인앱결제 릴리즈 노트', '원스토어 인앱결제 라이브러리 21.02.00 업데이트', '**원스토어 인앱결제 라이브러리 API V6(SDK V19) 출시*...

--- 순위 3 ---
관련 태그: ['purchaseId', 'purchaseToken', 'lastPurchaseId', 'acknowledgeState', 'purchaseState', 'lastPurchaseState']
내용: [title]: &#x20;Server API 변경
[section_path]: ['09. 원스토어 인앱결제 릴리즈 노트', '원스토어 인앱결제 라이브러리 21.02.00 업데이트', '**원스토어 인앱결제 라이브러리 API V6(SDK V19) 출시**', '&#x2...

